In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
data = pd.read_excel('card transactions.xlsx', parse_dates=['Date'])

In [55]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


## Data Cleaning

### Remove large transaction outliers / All but the P type transactions

In [56]:
data['z_Amount'] = (data['Amount'] - data['Amount'].mean()) / data['Amount'].std()

In [57]:
for i in range(7, 0, -1):
    print(i, (data['z_Amount'] >= i).sum())

7 1
6 1
5 1
4 2
3 2
2 27
1 72


In [58]:
data[data['z_Amount'] >= 4]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,z_Amount
47339,47340,5142275225,2010-06-22,NaN,INTERSOL,NaN,NaN,P,47900.00,0,4.744298
52714,52715,5142189135,2010-07-13,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0,309.971433


In [59]:
# drop outliers with amount that is 4 std away from mean
data = data.drop(index=[52714, 47339], columns=['z_Amount'])

In [60]:
data = data[data['Transtype'] == 'P']

In [61]:
data.Transtype.unique()

array(['P'], dtype=object)

### Missing Values

#### Merch State
- About 93% of records with missing state also do not have valid zip code:
 * For the other 7%, fill the state with the most frequent state within its zip code.
- If available, fill it with the last merch state associated with that Merch description, or first state after it.
- Fill it with the most frequent Merch state on that day.
- Notice that there are strange state names:
 * Some state are named as numbers.
 * I will just keep these numbers, because weird records may also be a sign of fraud.


In [62]:
data['Merch state'].isna().sum()

1019

In [63]:
na_state_zip = data[(data['Merch state'].isna()) & (data['Merch zip'].notna())]['Merch zip'].unique()
for zipcode in na_state_zip:
    temp_df = data[data['Merch zip'] == zipcode]
    if temp_df['Merch state'].isna().mean() != 1:
        target = temp_df['Merch state'].value_counts().index[0]
        data.loc[(data['Merch state'].isna()) & (data['Merch zip'] == zipcode), 'Merch state'] = target

In [64]:
data['Merch state'].isna().sum()

999

In [65]:
state_na_index = data[data['Merch state'].isna()].index
for i in state_na_index:
    des = data.loc[i, 'Merch description']
    notna_data = data[data['Merch state'].notna()]
    before = notna_data.loc[0:i-1, :]
    before = before[before['Merch description'] == des]
    if len(before) > 0:
        target = before.iloc[-1, :]['Merch state']
        data.loc[i, 'Merch state'] = target
    else:
        after = notna_data.loc[i:, :]
        after = after[after['Merch description'] == des]
        if len(after) > 0:
            target = after.iloc[-1, :]['Merch state']
            data.loc[i, 'Merch state'] = target

In [66]:
data['Merch state'].isna().sum()

334

In [67]:
na_state_date = data[data['Merch state'].isna()]['Date'].unique()
for date in na_state_date:
    temp_df = data[data['Date'] == date]
    target = temp_df['Merch state'].value_counts().index[0]
    data.loc[(data['Merch state'].isna())&(data['Date'] == date), 'Merch state'] = target

In [68]:
data['Merch state'].isna().sum()

0

#### Merch zip
- If available, fill it with the last Merch zip associated with that Merch num, or the first zip after it.
- If available, fill it with the last Merch zip associated with that Merch description, or the first zip after it.
- Fill it with the most frequent Merch zip within that state.
- Notice that there are some strange state:
 * These may be provinces in Canada.
 * I will just replace them with the a random zip code of each area that I found online. 
 * There are only 40 records, so this is okay.

In [69]:
data['Merch zip'].isna().sum()

4299

In [70]:
zip_na_index = data[data['Merch zip'].isna()].index
for i in zip_na_index:
    num = data.loc[i, 'Merchnum']
    notna_data = data[data['Merch zip'].notna()]
    before = notna_data.loc[0:i-1, :]
    before = before[before['Merchnum'] == num]
    if len(before) > 0:
        target = before.iloc[-1, :]['Merch zip']
        data.loc[i, 'Merch zip'] = target
    else:
        after = notna_data.loc[i:, :]
        after = after[after['Merchnum'] == num]
        if len(after) > 0:
            target = after.iloc[-1, :]['Merch zip']
            data.loc[i, 'Merch zip'] = target

In [71]:
data['Merch zip'].isna().sum()

2439

In [72]:
zip_na_index = data[data['Merch zip'].isna()].index
for i in zip_na_index:
    des = data.loc[i, 'Merch description']
    notna_data = data[data['Merch zip'].notna()]
    before = notna_data.loc[0:i-1, :]
    before = before[before['Merch description'] == des]
    if len(before) > 0:
        target = before.iloc[-1, :]['Merch zip']
        data.loc[i, 'Merch zip'] = target
    else:
        after = notna_data.loc[i:, :]
        after = after[after['Merch description'] ==  des]
        if len(after) > 0:
            target = after.iloc[-1, :]['Merch zip']
            data.loc[i, 'Merch zip'] = target

In [73]:
data['Merch zip'].isna().sum()

1994

In [74]:
na_zip_state = data[data['Merch zip'].isna()]['Merch state'].unique()
for state in na_zip_state:
    temp_df = data[data['Merch state'] == state]
    cnt = temp_df['Merch zip'].value_counts()
    if len(cnt) > 0:
        target = cnt.index[0]
        data.loc[(data['Merch zip'].isna())&(data['Merch state'] == state), 'Merch zip'] = target

In [75]:
data['Merch zip'].isna().sum()

40

In [76]:
# These may be provinces in Canada
# I will just replace them with the a random zip code of each area that I found online. 
# There are only 40 records, so this is okay.
# (Notice that these are zip codes of Canada)
data[data['Merch zip'].isna()]['Merch state'].unique()

array(['BC', 'QC', 'MB', 'NS', 'AB'], dtype=object)

In [77]:
mapper = {'BC': 'V5K0A1', 'QC': 'G1A0A2', 'MB': 'R0K0A0', 'NS': 'B3H0A2', 'AB': 'T0A0A0'}

In [78]:
states = data[data['Merch zip'].isna()]['Merch state'].unique()
for state in states:
    data.loc[(data['Merch zip'].isna()) & (data['Merch state'] == state), 'Merch zip'] = mapper[state]

In [79]:
data['Merch zip'].isna().sum()

0

#### Merchnum
- Merchnum = 0 may also be missing values
- If available, fill it with the last Merchnum associated with its description, or the first Merchnum appeared after it.
- Generate a new Merchnum for each Merch description (510 new numbers). The longest Merchnum is 13 digit, so I will start from a 14-digit number, 10000000000000.

In [80]:
data[data.Merchnum == '0'].shape

(53, 10)

In [81]:
data['Merchnum'] = data['Merchnum'].replace({'0': np.nan})
data[data.Merchnum == '0'].shape

(0, 10)

In [82]:
data.Merchnum.isna().sum()

3250

In [83]:
merchnum_na_index = data[data.Merchnum.isna()].index
for i in merchnum_na_index:
    des = data.loc[i, 'Merch description']
    notna_data = data[data['Merchnum'].notna()]
    before = notna_data.loc[0:i-1, :]
    before = before[before['Merch description'] == des]
    if len(before) > 0:
        target = before.iloc[-1, :]['Merchnum']
        data.loc[i, 'Merchnum'] = target
    else:
        after = notna_data.loc[i:, :]
        after = after[after['Merch description'] == des]
        if len(after) > 0:
            target = after.iloc[-1, :]['Merchnum']
            data.loc[i, 'Merchnum'] = target

In [84]:
data.Merchnum.isna().sum()

2093

In [85]:
des_list = data[data.Merchnum.isna()]['Merch description'].unique()
target = 10000000000000 
for des in des_list:
    data.loc[(data.Merchnum.isna())&(data['Merch description']==des), 'Merchnum'] = target
    target += 1

In [86]:
data.Merchnum.isna().sum()

0

In [88]:
data.isna().sum().sum()

0

In [89]:
# data.to_csv('cleaned_data.csv', index=False)